# Ejemplo 1: Tipo de Cambio del Bank of England (BoE)

Hacemos la carga de las librerias que vamos a utilizar

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

Acá vamos a definir el período de inicio y de finalización de nuestro scraping. 
La función today genera la fecha de hoy y la de timedelta permite elegir cuantos periodos hacia atras vamos a considerar (entendiendo a periodo como horas, dias, meses, años)

In [ ]:
fecha_inicio = datetime.today() - timedelta(days=30)

fecha_fin = datetime.today() - timedelta(days=1)

La funcion date_range de pandas nos va a permitir construir una lista de fechas al indicarle fecha de inicio y la fecha final. En este caso le pedimos que la frecuencia de los datos sean de lunes a viernes (B)

In [ ]:
rango_fechas = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='B')
rango_fechas

La pagina desde donde vamos a hacer el scraping es la siguiente:\
https://www.bankofengland.co.uk/boeapps/database/Rates.asp

y al seleccionar el 26 de enero, nos redirige a la siguiente URL:\
https://www.bankofengland.co.uk/boeapps/database/Rates.asp?TD=26&TM=Jan&TY=2021&into=GBP&rateview=D

Ahí podemos ver que cada valor despues del signo '**?**' se corresponde a nuestra elección en cada menú desplegable.

- El día se encuentra en TD=26

- El mes se encuentra en TM=Jan

- El año se encuentra en TY=2021

Podemos generar la lista de urls para después hacer las solicitudes a la página con las urls ya definidas. Para eso creamos una lista vacía y le vamos agregando cada una de las urls generadas con las especificaciones.
La función strftime permite que cada objeto fecha pase a ser un string con cierto especificación:
- %Y para año
- %b mes abreviado
- %d día

In [ ]:
urls = []

for fecha in rango_fechas:
    urls.append('https://www.bankofengland.co.uk/boeapps/database/Rates.asp?TD=' + 
         fecha.strftime('%d') +
        '&TM=' + fecha.strftime('%b') +
        '&TY=' + fecha.strftime('%Y') +
        '&into=GBP&rateview=D')
urls

Podemos tomar una de las urls generadas y verificar si pandas lee correctamente el contenido. Pandas puede leer csv, json, excel, xml y también formato html. La función read_html devuelve una lista de dataframes y en nuestro caso vamos a elegir el primer elemento ([0]) de esa lista.

In [ ]:
df = pd.read_html('https://www.bankofengland.co.uk/boeapps/database/Rates.asp?Travel=NIxAZx&into=GBP')[0]
df

Al ver que funciona correctamente en varias urls, podemos proceder a iterar sobre cada una de las direcciones y hacer la solicitud.
La idea detrás de este proceso de iteración es:
1. Hacer la solicitud con requests
2. Si la pagina existe, leemos el contenido con pandas y nos traemos el primer dataframe.
3. Eliminamos cualquier fila que tenga todos NaNs
4. Nos quedamos con las 2 primeras columnas (Moneda y tipo de cambio)
5. Renombramos las columnas y agregamos la fecha de ese día
6. Luego tener todo formateado, vamos agregando secuencialmente cada dataframe a uno definitivo.

In [ ]:
for indice, url in enumerate(urls):
    response = requests.get(url)
    if response.status_code == 200:
        df_temp = pd.read_html(response.content)[0]
        
        #Si ese día no hay datos (el dataframe no tiene filas)
        if df_temp.shape[0] != 0:
            df_temp.dropna(how='all', axis=1, inplace=True)

            #Mantengo solamente las 2 primeras columnas
            df_temp = df_temp.iloc[:, :2]

            df_temp.columns = ['Currency', 'Rate Exchange']
            df_temp['Date'] = pd.to_datetime(rango_fechas[indice].strftime('%Y-%m-%d'))

            if indice == 0:
                df = df_temp.copy()
            else:
                df = df.append(df_temp, ignore_index=True)
        else:
            pass
    else:
        pass

El resultado de nuestro scraping es el siguiente dataframe

In [ ]:
df